In [ ]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os



# 환경 설정
from dotenv import load_dotenv
load_dotenv()

prompt_template = ChatPromptTemplate([
    ('system', """
     
     # 당신은 주어진 json 데이터를 바탕으로 모델 파인튜닝용 jsonl 데이터를 생성하는 AI입니다.
     주어진 조건에 맞게 데이터를 생성하세요.
         
     # 기본 출력 형식
     {{
       "instruction": "",
       "input": "",
       "output": ""
     }}
          
     # 기본 규칙
     - 원본 데이터는 각 은행의 대출 상품의 데이터를 청크로 나눠둔 것입니다.
     - "instruction" 항목에는 질문 형식의 문장이, "output"에는 질문의 대답이 들어가야합니다.
     - "input" 항목은 비워 둡니다.
     - 각 항목의 content는 원본 source에서 나온 내용임을 알 수 있어야 합니다.
     (예시: "instrucrtion: 하나 청년 전세론의 가입 조건은?)
     - 주어진 데이터의 내용을 왜곡하거나 변경없이 조건에 맞춰 질문/대답 형식으로 양식에 맞춰 작성하세요.
          
     
     # 필수 규칙
     - 반드시 원본 데이터에 있는 내용에서 생성해야만 하며, 임의로 추가의 정보를 넣거나 생성하지 않습니다.
     - 각 항목의 내용을 질의 하나로 담기 힘들 경우 최대 3개의 질의로 나눠서 생성합니다.
     - 질문 혹은 답변에 source의 상품이름을 언급하여 어떤 상품의 정보인지 알수 있어야만 합니다.
    
    """),
    ('user', '{question}')
])

# 2. Model
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.3
)

# 3. OutputParser (StrOutputParser)
output_parser = StrOutputParser()

chain = prompt_template | model | output_parser


# 🔓 JSON 로딩
with open("./cleaned/cleaned_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)


output_path = "./training_data/qa_dataset.jsonl"
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
batch_size = 10

print("📁 저장될 파일 경로:", os.path.abspath(output_path))
print("📁 현재 작업 디렉토리:", os.getcwd())


for i in range(0, len(data), batch_size):
    batch = data[i:i+batch_size]
    
    for item in batch:
        source = item.get("source", "출처없음")
        content = item.get("content", "").strip()

        if not content:
            continue

        # 네 프롬프트에서 요구하는 question 포맷
        question = json.dumps({
            "source": source,
            "content": content
        }, ensure_ascii=False, indent=2)

        try:
            result = chain.invoke({"question": question})

            # ✅ 응답 전체 문자열 그대로 저장
            with open(output_path, "a", encoding="utf-8") as out_f:
                out_f.write(json.dumps(result, ensure_ascii=False) + "\n")

            print(f"✅ 답변 저장 완료 - {source}")

        except Exception as e:
            print(f"❌ 오류 발생 - {source}: {e}")


In [20]:
import json
import re

# 경로
input_path = "./training_data/qa_dataset.jsonl"
output_path = "./training_data/qa_dataset_cleaned.jsonl"



def remove_control_chars(s):
    return re.sub(r'[\x00-\x1F\x7F]', '', s)

blocks = []

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            # 🔧 순서 중요: ,{ 치환 먼저
            line = line.replace(",{", "{").strip()
            raw_json = json.loads(line)

            # 기본 이스케이프 처리
            raw_json = raw_json.replace("\\n", "\n").replace('\\"', '"').replace("\\", "").replace(".{", "{")
            block = ""
            depth = 0
            for char in raw_json:
                block += char
                if char == "{":
                    depth += 1
                elif char == "}":
                    depth -= 1
                    if depth == 0:
                        cleaned = remove_control_chars(block.strip())
                        blocks.append(cleaned)
                        block = ""
        except Exception:
            blocks.append(line.strip())

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    for block in blocks:
        f.write(block.strip())
        f.write("\n")

output_path

'./training_data/qa_dataset_cleaned.jsonl'